<a href="https://colab.research.google.com/github/longhowlam/python_hobby_stuff/blob/master/GTST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
import pandas as pd

gtst = pd.read_csv("/content/drive/My Drive/DataSets/GTST_Daily_data.csv")
gtst.sample(5)

,text_of_1month,datums,datums2
3031,Sjors raakt in paniek als ze zich realiseert ...,Vrijdag 15 december 2006,15 12 2006
1162,Jessica is er niet gelukkig mee dat de naam v...,Dinsdag 5 november 1996,5 11 1996
172,De politie staakt het zoeken naar Simon. Arni...,Woensdag 29 mei 1991,29 5 1991
3339,Ronja breekt in in de praktijk van Martijn en...,Vrijdag 21 november 2008,21 11 2008
3591,Lorena en Danny belanden met veel lol samen i...,Maandag 8 maart 2010,8 3 2010


In [19]:
gtst.datums2.str.lstrip()

0        1 10 1990
1        2 10 1990
2        3 10 1990
3        4 10 1990
4        5 10 1990
5        8 10 1990
6        9 10 1990
7       10 10 1990
8       11 10 1990
9       12 10 1990
10      15 10 1990
11      16 10 1990
12      17 10 1990
13      18 10 1990
14      19 10 1990
15      22 10 1990
16      23 10 1990
17      24 10 1990
18      25 10 1990
19      26 10 1990
20      29 10 1990
21      30 10 1990
22      31 10 1990
23       1 11 1990
24       2 11 1990
25       5 11 1990
26       6 11 1990
27       7 11 1990
28       8 11 1990
29       9 11 1990
           ...    
4060     20 4 2012
4061     23 4 2012
4062     24 4 2012
4063     25 4 2012
4064     26 4 2012
4065     27 4 2012
4066     30 4 2012
4067      1 5 2012
4068      2 5 2012
4069      3 5 2012
4070      4 5 2012
4071      7 5 2012
4072      8 5 2012
4073      9 5 2012
4074     10 5 2012
4075     11 5 2012
4076     14 5 2012
4077     15 5 2012
4078     16 5 2012
4079     17 5 2012
4080     18 5 2012
4081     21 

In [13]:
gtst.assign(datum = pd.to_datetime(gtst.datums2, format='%d-%m-%Y'))

ValueError: ignored